# 🌿 Plant Recognition with PlantCLEF 2025 on Kaggle

**Purpose**: Use PlantCLEF 2025 dataset directly on Kaggle without downloading 1TB locally

**Strategy**:
1. Run this notebook on Kaggle (datasets available there)
2. Accept user uploaded images via API
3. Use CLIP for similarity search
4. Return top-5 plant predictions
5. No need for local storage or IDrive2

**Dataset**: PlantCLEF 2025 (~1TB)  
**Model**: CLIP (openai/clip-vit-base-patch32)  
**Platform**: Kaggle Notebooks (GPU enabled)

## 1. Install and Import Required Libraries

In [ ]:
# Install required packages (run once on Kaggle)
!pip install -q transformers torch pillow

# Import libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

import torch
from transformers import CLIPProcessor, CLIPModel

print("✅ Libraries imported successfully")
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
print(f"💾 Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

## 2. Configure Kaggle Dataset Access

PlantCLEF 2025 dataset is available on Kaggle. We'll access it directly from `/kaggle/input/` directory.

In [ ]:
# Kaggle dataset path (automatically mounted on Kaggle notebooks)
DATASET_PATH = "/kaggle/input/plantclef2025"

# Check if dataset is available
if os.path.exists(DATASET_PATH):
    print("✅ PlantCLEF 2025 dataset found!")
    
    # List dataset contents
    contents = os.listdir(DATASET_PATH)
    print(f"\n📁 Dataset contents ({len(contents)} items):")
    for item in contents[:10]:
        print(f"   - {item}")
    if len(contents) > 10:
        print(f"   ... and {len(contents) - 10} more items")
else:
    print("❌ Dataset not found!")
    print("💡 Make sure to:")
    print("   1. Add 'PlantCLEF 2025' dataset to this notebook")
    print("   2. Enable GPU accelerator (Settings > Accelerator > GPU)")

## 3. Load CLIP Model

We'll use CLIP for zero-shot plant recognition and similarity search.

In [ ]:
# Load CLIP model and processor
print("🔄 Loading CLIP model...")
model_name = "openai/clip-vit-base-patch32"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

model.eval()  # Set to evaluation mode
print(f"✅ CLIP model loaded on {device}")
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Image Encoding Function

Function to encode images using CLIP with L2 normalization.

In [ ]:
def encode_image(image_path_or_pil):
    """
    Encode image using CLIP
    
    Args:
        image_path_or_pil: File path (str) or PIL Image
    
    Returns:
        numpy array: Normalized 512-dimensional embedding
    """
    # Load image
    if isinstance(image_path_or_pil, str):
        image = Image.open(image_path_or_pil).convert("RGB")
    else:
        image = image_path_or_pil.convert("RGB")
    
    # Process with CLIP
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Extract features
    with torch.no_grad():
        features = model.get_image_features(**inputs)
        # L2 normalization for cosine similarity
        features = features / features.norm(dim=-1, keepdim=True)
    
    return features.cpu().numpy().flatten()

# Test encoding function
print("✅ Image encoding function ready")

## 5. User Image Upload and Processing

**API Integration**: This notebook will be called from FastAPI backend.

When a user uploads an image:
1. Backend sends image to this Kaggle notebook (via Kaggle API)
2. Notebook processes image with CLIP
3. Searches PlantCLEF dataset for similar plants
4. Returns top-5 predictions

In [ ]:
# Example: Process user uploaded image
# In production, this will be called via Kaggle API

def process_user_image(image_input):
    """
    Process user uploaded image and find similar plants
    
    Args:
        image_input: PIL Image or file path
    
    Returns:
        dict: Predictions with confidence scores
    """
    print("🔄 Processing user image...")
    
    # Encode user image
    user_embedding = encode_image(image_input)
    print(f"✅ Image encoded: {user_embedding.shape}")
    
    # TODO: Load pre-computed embeddings from PlantCLEF dataset
    # For now, return mock predictions
    predictions = [
        {"species": "Rosa damascena", "confidence": 0.95, "family": "Rosaceae"},
        {"species": "Rosa gallica", "confidence": 0.89, "family": "Rosaceae"},
        {"species": "Rosa canina", "confidence": 0.82, "family": "Rosaceae"},
        {"species": "Hibiscus rosa-sinensis", "confidence": 0.76, "family": "Malvaceae"},
        {"species": "Camellia japonica", "confidence": 0.71, "family": "Theaceae"}
    ]
    
    return {
        "success": True,
        "predictions": predictions,
        "embedding_dim": len(user_embedding),
        "processing_time": "~2.5s"
    }

print("✅ Image processing function ready")

## 6. Visualization (Optional)

Display sample images from PlantCLEF dataset.

In [ ]:
# Visualize sample images from dataset
def visualize_samples(dataset_path, num_samples=6):
    """Display random samples from PlantCLEF dataset"""
    if not os.path.exists(dataset_path):
        print("⚠️ Dataset not found - skipping visualization")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # TODO: Load actual images from PlantCLEF
    # For now, show placeholder
    for i, ax in enumerate(axes):
        ax.text(0.5, 0.5, f"PlantCLEF Image {i+1}\n(Load from dataset)", 
                ha='center', va='center', fontsize=12)
        ax.axis('off')
    
    plt.suptitle("Sample Plants from PlantCLEF 2025 Dataset", fontsize=16)
    plt.tight_layout()
    plt.show()

print("✅ Visualization function ready")

## 7. Next Steps

**To complete this notebook**:

1. **Add PlantCLEF 2025 dataset** to this Kaggle notebook
   - Go to "Add data" → Search "PlantCLEF 2025"
   - Add to notebook

2. **Pre-compute embeddings** (one-time setup)
   - Encode all PlantCLEF images with CLIP
   - Save embeddings to disk (~2GB for 1M images)
   - Load embeddings for fast similarity search

3. **Implement similarity search**
   - Use cosine similarity to find k-nearest neighbors
   - Return top-5 predictions with confidence scores

4. **API Integration**
   - Set up Kaggle API endpoint
   - Accept images from FastAPI backend
   - Return predictions as JSON

**Benefits of this approach**:
- ✅ No need to download 1TB dataset locally
- ✅ No IDrive2 storage needed
- ✅ Use Kaggle's free GPU for inference
- ✅ Access latest PlantCLEF updates directly